# Notebook for estimating stress/threat from water withdrawals to GDEs

Consolidate well level trends and water rights data around GDEs.

Requires: Water level data from NWIS (USGS) and WellNet (NDWR) trends calculated in GDE_threats_level_trends.R, and water rights data from NDWR hydro abstracts.



## Set up environments in ArcGIS Pro

In [ ]:
# Import ArcGIS modules and check out spatial analyst extension
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

In [ ]:
# Path to temporary geodatabase
#path =  r"folder_path\gdbname.gdb"
path =  r"K:\GIS3\Projects\GDE\Maps\GDE_Threats\GDE_Threats.gdb"

# Environment settings
env.workspace = path
env.overwriteOutput = True
env.outputCoordinateSystem = arcpy.SpatialReference(26911) # Spatial reference NAD 1983 UTM Zone 11N. The code is '26911'

## Read in groundwater and basin status data

In [ ]:
# Groundwater levels (trends)
#gw = r'path_to_folder\gwlevels_stats_110321.shp'
gw = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\gwlevels_stats_110321.shp'
print([f.name for f in arcpy.ListFields(gw)])

# OPTIONAL Groundwater levels - filtered to well with 20 years of pre-irrigation-season data
#gw = r'path_to_folder\gwlevels_newrules_051021.shp'


In [ ]:
# Basin appropriation/pumping status
# Wilson 2020: https://tnc.box.com/s/ns19ih9cka3g4at8qrrcs4j314mmp74c
#basins = r'path_to_folder\hydrographic_basin_boundaries.shp'
basins = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NDWR\hydrographic_basin_boundaries.shp'
print([f.name for f in arcpy.ListFields(basins)])

## Create a copy of the GDE geodatabase to store stress/threat data

*Note* - Phretophyte communities layer needs to be "exploded". There are several very large multipart features, but these should be made into singlepart features for better analysis. This geodatabase will be larger than the primary Nevada iGDE geodatabase.

In [ ]:
# Copy GDE database layers to a new GDB - attribute these ones
# gde2 = arcpy.Copy_management(r'path_to_folder\NV_iGDE_022120.gdb', 
#                              r'path_to_folder\NV_iGDE_assess.gdb')
# arcpy.ListFeatureClasses(r'path_to_folder\NV_iGDE_assess.gdb')


gde2 = arcpy.Copy_management(r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_022120.gdb', 
                             r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb')
arcpy.ListFeatureClasses(r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb')

In [ ]:
# GDE layers
# springs = r'path_to_folder\NV_iGDE_assess.gdb\Springs'
# wet = r'path_to_folder\NV_iGDE_assess.gdb\Wetlands'
# phr = r'path_to_folder\NV_iGDE_assess.gdb\Phreatophytes' #**need to explode**#
# lp = r'path_to_folder\NV_iGDE_assess.gdb\Lakes_Playas'
# rs = r'path_to_folder\NV_iGDE_assess.gdb\Rivers_Streams'

springs = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Springs'
wet = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Wetlands'
phr = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Phreatophytes' #**need to explode**#
lp = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Lakes_Playas'
rs = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Rivers_Streams'



In [ ]:
# Explode Phretophytes so each one can be attributed separately
phr = arcpy.MultipartToSinglepart_management(phr, r'path_to_folder\NV_iGDE_assess.gdb\Phreatophytes_Explode')
print(arcpy.GetCount_management(phr))
phr = r'path_to_folder\NV_iGDE_assess.gdb\Phreatophytes_Explode'
print([f.name for f in arcpy.ListFields(phr)])


## Threats to GDEs from declining groundwater levels

If a GDe falls within a half mile of a shallow groundwater area (groundwater is within 50 feet of the surface) according to Lopes et al 2006.

Shallow groundwater areas were digitized from the Lopes report map.

Attribute: __Shallow_GW = 0 (no threat) or 1 (threat)__

In [ ]:
# Shallow groundwater polygon laeyr
shallow = r'path_to_folder\GDE_Threats.gdb\shallow_gw_dissolve'
shallow = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Maps\GDE_Threats.gdb\shallow_gw_dissolve'

In [ ]:
# For each GDE type - attribute whether it falls within half-mile of shallow gw
# Springs
arcpy.AddField_management(springs, 'Shallow_GW', 'LONG')
[f.name for f in arcpy.ListFields(springs)]
spr_select = arcpy.SelectLayerByLocation_management(springs, 'INTERSECT', shallow)
arcpy.CalculateField_management(spr_select, 'Shallow_GW', 1)

# Wetlands
arcpy.AddField_management(wet, 'Shallow_GW', 'LONG')
[f.name for f in arcpy.ListFields(wet)]
wet_select = arcpy.SelectLayerByLocation_management(wet, 'INTERSECT', shallow)
arcpy.CalculateField_management(wet_select, 'Shallow_GW', 1)

# Phreatophytes
arcpy.AddField_management(phr2, 'Shallow_GW', 'LONG')
[f.name for f in arcpy.ListFields(phr)]
phr_select = arcpy.SelectLayerByLocation_management(phr, 'INTERSECT', shallow)
arcpy.CalculateField_management(phr_select, 'Shallow_GW', 1)

# Lakes/playas
arcpy.AddField_management(lp, 'Shallow_GW', 'LONG')
[f.name for f in arcpy.ListFields(lp)]
lp_select = arcpy.SelectLayerByLocation_management(lp, 'INTERSECT', shallow)
arcpy.CalculateField_management(lp_select, 'Shallow_GW', 1)

# Rivers/streams
arcpy.AddField_management(rs, 'Shallow_GW', 'LONG')
[f.name for f in arcpy.ListFields(rs)]
rs_select = arcpy.SelectLayerByLocation_management(rs, 'INTERSECT', shallow)
arcpy.CalculateField_management(rs_select, 'Shallow_GW', 1)


## Stress to GDEs if near a well with a significantly declining groundwater level

If GDEs fall within a half-mile of wells with significant declining groundwater levels (p-value <= 0.05 and negative Sens_Slope value), they are considered stressed.

For polygon GDE features, stress applies if any portion of the polygon falls within a half mile of selected wells. 

Not attributing with specific stressor/threat data. I.e. using this index, we can tell if a GDE is stressed, but don't know the exact trend of the well it is getting stressed by. Need to go to source data (NDWR/NWIS trend sites) for that information.

Attribute: __Falling_GW = 0 (not stressed) or 1 (stressed)__

In [ ]:
# Half mile buffer around falling gw levels
gw_fall = arcpy.SelectLayerByAttribute_management(gw, 'NEW_SELECTION', "SigFall = 1")
arcpy.GetCount_management(gw_fall)
gw_buff = arcpy.Buffer_analysis(gw_fall, 'gw_falling_halfmile', '0.5 Mile', '', '', 'ALL')

gw_buff = r'gw_falling_halfmile'
print(arcpy.GetCount_management(gw_buff))

In [ ]:
# Select and attribute GDEs, as above
# Springs
arcpy.AddField_management(springs, 'Falling_GW', 'LONG')
[f.name for f in arcpy.ListFields(springs)]
spr_select = arcpy.SelectLayerByLocation_management(springs, 'INTERSECT', gw_buff)
arcpy.CalculateField_management(spr_select, 'Falling_GW', 1)

# Wetlands
arcpy.AddField_management(wet, 'Falling_GW', 'LONG')
[f.name for f in arcpy.ListFields(wet)]
wet_select = arcpy.SelectLayerByLocation_management(wet, 'INTERSECT', gw_buff)
arcpy.CalculateField_management(wet_select, 'Falling_GW', 1)

# Phreatophytes
arcpy.AddField_management(phr2, 'Falling_GW', 'LONG')
[f.name for f in arcpy.ListFields(phr2)]
phr_select = arcpy.SelectLayerByLocation_management(phr2, 'INTERSECT', gw_buff)
arcpy.CalculateField_management(phr_select, 'Falling_GW', 1)

# Lakes/playas
arcpy.AddField_management(lp, 'Falling_GW', 'LONG')
[f.name for f in arcpy.ListFields(lp)]
lp_select = arcpy.SelectLayerByLocation_management(lp, 'INTERSECT', gw_buff)
arcpy.CalculateField_management(lp_select, 'Falling_GW', 1)

# Rivers/streams
arcpy.AddField_management(rs, 'Falling_GW', 'LONG')
[f.name for f in arcpy.ListFields(rs)]
rs_select = arcpy.SelectLayerByLocation_management(rs, 'INTERSECT', gw_buff)
arcpy.CalculateField_management(rs_select, 'Falling_GW', 1)

## Threat or Stress to GDEs from basin overcommitment/overpumping
 
 GDEs are considered stressed if basin is being overpumped.

 Attribute: __PumpStr = 0 (not stressed) or 1 (stressed)__

 GDEs are considered threatened if the basins are overcommitted (commited water rights v. pereninal yield). Different levels of commitment of perennial yield are given different threat values.

 Attribute: __CommitThr = 0 (no threat), 0.5 (moderate threat), or 1 (threat)__

In [ ]:
# Springs
spr_commit = arcpy.SpatialJoin_analysis(springs, basins, 'temp_springs_join')
[f.name for f in arcpy.ListFields(spr_commit)]
arcpy.GetCount_management(spr_commit)
arcpy.GetCount_management(springs)

arcpy.AddField_management(spr_commit, 'CommitThr', 'DOUBLE')
with arcpy.da.UpdateCursor(spr_commit, ['OverCommit', 'CommitThr']) as cursor:
    for row in cursor:
        if row[0] == 100:
            row[1] = 0.5
            cursor.updateRow(row)
        elif row[0] == 200:
            row[1] = 1
            cursor.updateRow(row)
        elif row[0] == 0:
            row[1] = 0
            cursor.updateRow(row)
del cursor

arcpy.AddField_management(spr_commit, 'PumpStr', 'DOUBLE')
arcpy.CalculateField_management(spr_commit, 'PumpStr', '!OverPump!', 'PYTHON3')

arcpy.JoinField_management(springs, 'OBJECTID', spr_commit, 'OBJECTID', ['CommitThr', 'PumpStr']) # Also joining overpump (stress)
[f.name for f in arcpy.ListFields(springs)]
# Manually deleted/edited some springs that fall just on the border or outside the basin boundaries


In [ ]:
# Phreatophytes
phr_commit = arcpy.SpatialJoin_analysis(phr, basins, 'temp_phr_join')
[f.name for f in arcpy.ListFields(phr_commit)]
arcpy.GetCount_management(phr_commit)
arcpy.GetCount_management(phr)

arcpy.AddField_management(phr_commit, 'CommitThr', 'DOUBLE')
with arcpy.da.UpdateCursor(phr_commit, ['OverCommit', 'CommitThr']) as cursor:
    for row in cursor:
        if row[0] == 100:
            row[1] = 0.5
            cursor.updateRow(row)
        elif row[0] == 200:
            row[1] = 1
            cursor.updateRow(row)
        elif row[0] == 0:
            row[1] = 0
            cursor.updateRow(row)
del cursor

arcpy.AddField_management(phr_commit, 'PumpStr', 'DOUBLE')
arcpy.CalculateField_management(phr_commit, 'PumpStr', '!OverPump!', 'PYTHON3')

arcpy.JoinField_management(phr, 'OBJECTID', phr_commit, 'OBJECTID', ['CommitThr', 'PumpStr']) # Also joining overpump (stress)
[f.name for f in arcpy.ListFields(phr)]


In [ ]:
# Wetlands
wet_commit = arcpy.SpatialJoin_analysis(wet, basins, 'temp_wet_join')
[f.name for f in arcpy.ListFields(wet_commit)]
arcpy.GetCount_management(wet_commit)
arcpy.GetCount_management(wet)

arcpy.AddField_management(wet_commit, 'CommitThr', 'DOUBLE')
with arcpy.da.UpdateCursor(wet_commit, ['OverCommit', 'CommitThr']) as cursor:
    for row in cursor:
        if row[0] == 100:
            row[1] = 0.5
            cursor.updateRow(row)
        elif row[0] == 200:
            row[1] = 1
            cursor.updateRow(row)
        elif row[0] == 0:
            row[1] = 0
            cursor.updateRow(row)
del cursor

arcpy.AddField_management(wet_commit, 'PumpStr', 'DOUBLE')
arcpy.CalculateField_management(wet_commit, 'PumpStr', '!OverPump!', 'PYTHON3')

arcpy.JoinField_management(wet, 'OBJECTID', wet_commit, 'OBJECTID', ['CommitThr', 'PumpStr']) # Also joining overpump (stress)
[f.name for f in arcpy.ListFields(wet)]


In [ ]:
# Rivers/streams
rs_commit = arcpy.SpatialJoin_analysis(rs, basins, 'temp_rs_join')
[f.name for f in arcpy.ListFields(rs_commit)]
arcpy.GetCount_management(rs_commit)
arcpy.GetCount_management(rs)

arcpy.AddField_management(rs_commit, 'CommitThr', 'DOUBLE')
with arcpy.da.UpdateCursor(rs_commit, ['OverCommit', 'CommitThr']) as cursor:
    for row in cursor:
        if row[0] == 100:
            row[1] = 0.5
            cursor.updateRow(row)
        elif row[0] == 200:
            row[1] = 1
            cursor.updateRow(row)
        elif row[0] == 0:
            row[1] = 0
            cursor.updateRow(row)
del cursor

arcpy.AddField_management(rs_commit, 'PumpStr', 'DOUBLE')
arcpy.CalculateField_management(rs_commit, 'PumpStr', '!OverPump!', 'PYTHON3')

arcpy.JoinField_management(rs, 'OBJECTID', rs_commit, 'OBJECTID', ['CommitThr', 'PumpStr']) # Also joining overpump (stress)
[f.name for f in arcpy.ListFields(rs)]



In [ ]:
# Lakes/playas
lp_commit = arcpy.SpatialJoin_analysis(lp, basins, 'temp_lp_join')
[f.name for f in arcpy.ListFields(lp_commit)]
arcpy.GetCount_management(lp_commit)
arcpy.GetCount_management(lp)

arcpy.AddField_management(lp_commit, 'CommitThr', 'DOUBLE')
with arcpy.da.UpdateCursor(lp_commit, ['OverCommit', 'CommitThr']) as cursor:
    for row in cursor:
        if row[0] == 100:
            row[1] = 0.5
            cursor.updateRow(row)
        elif row[0] == 200:
            row[1] = 1
            cursor.updateRow(row)
        elif row[0] == 0:
            row[1] = 0
            cursor.updateRow(row)
del cursor

arcpy.AddField_management(lp_commit, 'PumpStr', 'DOUBLE')
arcpy.CalculateField_management(lp_commit, 'PumpStr', '!OverPump!', 'PYTHON3')

arcpy.JoinField_management(lp, 'OBJECTID', lp_commit, 'OBJECTID', ['CommitThr', 'PumpStr']) # Also joining overpump (stress)
[f.name for f in arcpy.ListFields(lp)]
